# Assignment 1: Auto-Correct

_See [Assignment 1: Auto-correct](https://sikoried.github.io/sequence-learning/01/autocorrect/)._

## String Distances

Gemischtes Doppel 1 | Gemischtes Doppel 2 | Gemischtes Doppel 3
-|-|-
![Gemischtes Doppel 1](res/gem_doppel_1.jpg) | ![Gemischtes Doppel 2](res/gem_doppel_2.jpg) | ![Gemischtes Doppel 3](res/gem_doppel_2.jpg)

In the first part of the exercise, we will compute the Hamming and edit distances for the string pairs above (source: Gemischten Doppel, Süddeutsche).
Let's start with the simpler one: [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance).
Since this distance is only defined for strings of equal length; for your implementation, make a reasonable modification to support different lengths

In [ ]:
import sys

gem_doppel = [
    ("GCGTATGAGGCTAACGC", "GCTATGCGGCTATACGC"),
    ("kühler schrank", "schüler krank"),
    ("the longest", "longest day"),
    ("nicht ausgeloggt", "licht ausgenockt"),
    ("gurken schaben", "schurkengaben")
]

In [ ]:
def hamming(x, y):
    # TODO compute the hamming distance
    return 0

for (a, b) in gem_doppel:
    print("hamming('%s', '%s') = %d" % (a, b, hamming(a, b)))


# hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
# hamming('kühler schrank', 'schüler krank') = 13
# hamming('the longest', 'longest day') = 11
# hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
# hamming('gurken schaben', 'schurkengaben') = 14

As you can see, the Hamming distances are quite large, since only viable "operations" are _match_ (no cost) and _replace_ (cost 1). For a more nuanced measure, implement the [edit distance](https://en.wikipedia.org/wiki/Edit_distance) also allows for insertions and deletions. Make sure to make the cost of those operations configurable.

_Hint:_ This is a good opportunity to familiarize yourself with [`numpy`](https://numpy.org/) and its matrices and range operators; we'll use those throughout the semester.

In [ ]:
import numpy as np

def edit(x, y, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}):
    # TODO compute the edit distance with respect to the given costs
    return 0


for (a, b) in gem_doppel:
    print("edit('%s', '%s') = %d" % (a, b, edit(a, b)))

    
# edit('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3
# edit('kühler schrank', 'schüler krank') = 6
# edit('the longest', 'longest day') = 8
# edit('nicht ausgeloggt', 'licht ausgenockt') = 4
# edit('gurken schaben', 'schurkengaben') = 7

As you can see, the edit distances relate much better to the similarity of the strings, but they still don't really tell us where and how the strings differ.
Extend your implementation from above by also computing a backtrace of the operations which can be used to print the alignment of the two strings.

In [ ]:
def edit2(x, y, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}):
    D = np.zeros((len(x) + 1, len(y) + 1), dtype=int)

    # for the empty word, costs match the length of the other string
    D[0, 1:] = range(1, len(y) + 1)
    D[1:, 0] = range(1, len(x) + 1)

    # this array will hold the journal of operations for backtracking
    T = np.zeros((len(x) + 1, len(y) + 1), dtype=np.object_)
    T[0, 0] = 'ε'
    T[0, 1:] = 'i'
    T[1:, 0] = 'd'
    
    # compute the trace from T and return the alignment string
    return D[len(x), len(y)], '???'


# test it, output some useful visualizations of the alignment strings
for (a, b) in gem_doppel:
    d, tr = edit2(a, b)
    print("edit('%s', '%s') = %d (%s)" % (a, b, d, tr))

    for i, op in enumerate(tr):
        if op == 'i':
            a = a[:i] + '-' + a[i:]
        if op == 'd':
            b = b[:i] + '-' + b[i:]

    print('  ' + a)
    print('  ' + tr.replace('m', ' ').replace('s', '*'))
    print('  ' + b)

# edit('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
#   GCGTATGAGGCTA-ACGC
#     d    *     i    
#   GC-TATGCGGCTATACGC
# edit('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
#   küh-ler schrank
#   ** i    *dd    
#   schüler k--rank
# edit('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
#   the longest----
#   dddd       iiii
#   ----longest day
# edit('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
#   nicht ausgeloggt
#   *          * ** 
#   licht ausgenockt
# edit('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)
#   g--urken schaben
#   *ii     *ddd    
#   schurkeng---aben

## Spelling Correction

For spelling correction, we will use prior knowledge, to put some learning into our system.
The underlying idea is the Noisy Channel Model, that is: The user intends to write a word w, but through some noise in the process, happens to type the word x.

The correct word ŵ  is that word, that is a valid candidate and has the highest probability:

$$
\begin{eqnarray}
\DeclareMathOperator*{\argmax}{argmax}
\hat{w} & = & \argmax_{w \in V} P(w | x) \\
        & = & \argmax_{w \in V} \frac{P(x|w) P(w)}{P(x)} \\
        & = & \argmax_{w \in V} P(x|w) P(w)
\end{eqnarray}
$$

- The candidates V can be obtained from a vocabulary.
- The probability $P(w)$ of a word w can be learned (counted) from data.
- The probability $P(x|w)$ is more complicated... It could be learned from data, but we could also use a heuristic that relates to the edit distance, e.g. rank by distance.

You can find word statistics and training data at: <http://norvig.com/ngrams/> (The single word counts are part of this repo).

### Further Reading

- <http://norvig.com/spell-correct.html>
- Mays, Eric, Fred J. Damerau and Robert L. Mercer. 1991. Context based spelling correction. _Information Processing and Management,_ 23(5), 517–522. (IBM)
- Kernighan, Mark D., Kenneth W. Church, and William A. Gale. 1990. A spelling correction program based on a noisy channel model. _Proceedings of COLING 1990,_ 205-210. (Bell Labs)

### Step 1: Read in vocabulary and counts

In [ ]:
# contains lines of "word <count>"
counts_fn = 'data/count_1w.txt.gz'

# read in vocabulary: str -> count
voc = {}

import gzip
with gzip.open(counts_fn, "rb") as f:
    for line in f:
        # TODO
        pass

for (k, v) in voc:
    # TODO normalize the counts
    pass

print("Read in %d lemmas." % len(voc))

### Step 2: Baseline implementation

Implement a (pretty inefficient) spell corrector that, for a given word `w`, suggests at most `max_cand=5` candidate words.
To speed up the computation a little bit, consider only words that differ at most `max_dist=3` in length.

In [ ]:
from operator import itemgetter

# suggest a list of candidates for the entered word
def suggest(w, max_cand=5, max_dist=3):
    # TODO
    # nb: if the word is an exact hit in the vocab, return just that word


    # make sure the list is just max_cand long...
    return []


examples = [
    "pirates",      # in-voc
    "pirutes",      # pirates?
    "continoisly",  # continuosly?
]

for w in examples:
    print(w, suggest(w, max_cand=3))
    

# pirates [('pirates', 0, -11.408058827802126)]
# pirutes [('pirates', 1, -11.408058827802126), ('minutes', 2, -8.717825438953103), ('viruses', 2, -11.111468702571859)]
# continoisly [('continously', 1, -15.735337826575178), ('continuously', 2, -11.560071979871001), ('continuosly', 2, -17.009283000138204)]

### Step 3: Better Heuristic

Let's use a more sophisticated heuristic, that doesn't sort the data twice, but combines the distance with the relative frequency.
Here's the gist of it: while the distances are (typically) 0, 1, 2, 3..., the relative frequencies are very small numbers.

To model the Bayesian rule above, we need two quantities:

- $P(w)$: this is just the relative frequency
- $P(x|w)$: let's assume that about 1/3 of the time, we're just one symbol off; 1/2 for two, etc. (These don't really form probabilities, but we just want something probability-like :-)
- we may want to balance those quantities, since they might be orders of magnitude different

Mathematically speaking, we want something like

$$
\hat{w} = \argmax_{w \in V} = P(x|w) * P(w)^\beta \quad .
$$

For numerical reasons, we can apply the `log` and discard factors that are equal for all words:

$$
\begin{eqnarray}
\DeclareMathOperator*{\argmax}{argmax}
\hat{w} & = & \argmax_{w \in V} \log(P(x|w)) + \beta \log(P(w)) \\
        & = & \argmax_{w \in V} \left[ -\log(\frac{1}{2 + \text{edit}(w, x)}) + \beta \log \frac{\text{count}(w)}{\sum_x \text{count}(x)} \right] \\
        & = & \argmax_{w \in V} \left[ \beta \log \text{count}(w) - \log\left(2 + \text{edit}(w, x)\right) \right]
\end{eqnarray}
$$

In [ ]:
# TODO reload the voc if necessary

In [ ]:
def suggest2(w, beta, max_cand=5, max_dist=3, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}):
    # suggest at most max_cand words, using the above heuristig
    return []
  

examples = [
    "pirates",    # in-voc
    "pirutes",    # pirates?
    "continoisly",  # continuosly?
]

for w in examples:
    print(w, suggest2(w, 0.1, max_cand=3))

# pirates [('pirates', 0.6931471805599453, 1.569214519355234)]
# pirutes [('pirates', 1.0986122886681098, 1.569214519355234), ('minutes', 1.3862943611198906, 1.8382378582401362), ('prices', 1.6094379124341003, 1.9310363514927111)]
# continoisly [('continuously', 1.3862943611198906, 1.5540132041483465), ('continously', 1.0986122886681098, 1.1364866194779288), ('continuity', 1.6094379124341003, 1.561483500710584)]

## Efficient Implementation

Use a prefix tree to efficiently compute the edit distance for a large number of words.

In [ ]:
class PrefixTree:
    # TODO write a basic prefix tree implementation
    # nb: you may want to add __str__, __repr__ and a to_string_lines method to debug it...
    def __init__(self, parent, prefix):
        pass
    
    def size(self):
        pass
    
    def insert(self, word):
        pass

    def query(self, word):
        pass


# read all entries into the prefix tree
root = PrefixTree(None, 'ε')

# TODO read vocab and store in prefix tree

# you may want to debug this...
root.insert('haus', 1)
root.insert('habe', 1)
root.insert('hau', 1)
root.insert('auto', 1)
root.insert('autark', 1)

print("Indexed %d words" % root.size())

In [ ]:
def edit3(root, word, max_dist=3, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}):
    # efficient implementation of edit distance for larg vocabulary stored in orefix tree
    # nb: return hits as dict: word -> edit-dist
    return {}


print(edit3, 'hans')

# {haus: 1, habe: 2, ...}

In [ ]:
# benchmark your examples
for e in examples:
    print(e)
    
    # old-school
    %time base = [(w, edit(e, w)) for w in root.voc]
    
    # faster implementation
    %time faster = edit3(root, e)